# Eigenvalue problems

We solve the generalized eigenvalue problem
$$A u=\lambda M u$$,

where A comes from $\int \nabla u \nabla v$, and $M$ from $\int uv$, on the space $H_0^1$

In [1]:
import netgen.gui
%gui tk
from netgen.geom2d import unit_square
from ngsolve import *
from math import pi
import scipy.linalg
from scipy import random 
from random import random

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))


We setup a stiffness matrix A and a mass matrix M, and declare a preconditioner for A:

In [2]:
fes = H1(mesh, order=4, dirichlet=[1,2,3,4])
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v))
pre = Preconditioner(a, "multigrid")

m = BilinearForm(fes)
m += SymbolicBFI(u*v)

a.Assemble()
m.Assemble()

u = GridFunction(fes)

The inverse iteration is
$$u_{n+1}=A^{−1}Mu_n,$$

where the Rayleigh quotient
$$\rho_n=\frac{\langle Au_n,u_n \rangle}{\langle Mu_n,u_n\rangle}$$

converges to the smallest eigen-value, with rate of convergence $\lambda_1/\lambda_2$.

The preconditioned inverse iteration (PINVIT), see [Knyazef+Neymeyr], replaces $A^{−1}$
by an approximate inverse $C^{−1}$

$$\rho_n=\frac{\langle Au_n,u_n\rangle}{\langle Mu_n,u_n\rangle}$$

$$w_n=C^{−1}(Au_n−\rho Mu_n)$$
$$u_{n+1}=u_n+\alpha w_n$$

The optimal step-size $\alpha$ is found by minimizing the Rayleigh-quotient on a two-dimensional space:
$$u_{n+1}=\arg \min_{v\in \lbrace u_n,w_n\rbrace} \frac{\langle Av,v\rangle}{\langle Mv,v\rangle}$$

This minimization problem can be solved by a small eigenvalue problem
$$ay=\lambda my$$

with matrices
$$a = \left(
\begin{array}[cc]
\ \langle Au_n,u_n\rangle & \langle Au_n,w_n\rangle\\
\langle Aw_n,u_n\rangle & \langle Aw_n,w_n\rangle
\end{array}
\right) \hspace{3em} m = \left(
\begin{array}[cc]
\ \langle Mu_n,u_n\rangle & \langle Mu_n,w_n\rangle\\
\langle Mw_n,u_n\rangle & \langle Mw_n,w_n\rangle
\end{array}
\right) $$

Then, the new iterate is

$$u_{n+1}=y_1 u_n+y_2 w_n$$

where $y$ is the eigenvector corresponding to the smaller eigenvalue.
### Implementation in NGSolve

Create some help vectors:


In [3]:
r = u.vec.CreateVector()
w = u.vec.CreateVector()
Mu = u.vec.CreateVector()
Au = u.vec.CreateVector()
Mw = u.vec.CreateVector()
Aw = u.vec.CreateVector()


random initial condition, zero on the boundary

In [5]:
freedofs = fes.FreeDofs()    
for i in range(len(u.vec)):
    u.vec[i] = random() if freedofs[i] else 0   

In [6]:
for i in range(20):
    Au.data = a.mat * u.vec
    Mu.data = m.mat * u.vec
    auu = InnerProduct(Au, u.vec)
    muu = InnerProduct(Mu, u.vec)
    # Rayleigh quotiont
    lam = auu/muu
    print (lam / (pi**2))
    # residual
    r.data = Au - lam * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = a.mat * w
    Mw.data = m.mat * w

    # setup and solve 2x2 small eigenvalue problem
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)
    
    
    eval,evec = scipy.linalg.eigh(asmall, b=msmall)
    # print (eval / pi**2)
    # print (evec)
    u.vec.data = float(evec[0,0]) * u.vec + float(evec[1,0]) * w
    

24.59133342199071
2.050274065207008
2.0014306792546024
2.000207345153818
2.0000469873405735
2.0000119557546174
2.0000031174313375
2.0000008235789197
2.0000002195363495
2.000000058928998
2.0000000159200018
2.0000000043406105
2.000000001209653
2.000000000360156
2.0000000001290097
2.000000000065966
2.0000000000487366
2.00000000004402
2.000000000042728
2.0000000000423728



### Simultaneous iteration for several eigenvalues

Declare GridFunction with multiple components to store several eigenvectors.


In [8]:
num = 5
u = GridFunction(fes, multidim=num)


Create list of help vectors:


In [9]:
r = u.vec.CreateVector()
Av = u.vec.CreateVector()
Mv = u.vec.CreateVector()

vecs = []
for i in range(2*num):
    vecs.append (u.vec.CreateVector())

freedofs = fes.FreeDofs()    
for v in u.vecs:
    for i in range(len(u.vec)):
        v[i] = random() if freedofs[i] else 0



Compute $num$ residuals, and solve small eigenvalue problem on $2num$-dimensional space

In [10]:
lams = num * [1]

asmall = Matrix(2*num, 2*num)
msmall = Matrix(2*num, 2*num)

for i in range(100):
    
    for j in range(num):
        vecs[j].data = u.vecs[j]
        r.data = a.mat * vecs[j] - lams[j] * m.mat * vecs[j]
        # r.data = 1/Norm(r) * r
        r *= 1/Norm(r)
        vecs[num+j].data = pre.mat * r

    for j in range(2*num):
        Av.data = a.mat * vecs[j]
        Mv.data = m.mat * vecs[j]
        for k in range(2*num):
            asmall[j,k] = InnerProduct(Av, vecs[k])
            msmall[j,k] = InnerProduct(Mv, vecs[k])

    ev,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    lams[0:num] = ev[0:num]
    print (i, ":", [lam/pi**2 for lam in lams])
    
    for j in range(num):
        r[:] = 0.0
        for k in range(2*num):
            r.data += float(evec[k,j]) * vecs[k]
        u.vecs[j].data = r
    
Draw (u)

0 : [11.898213866144609, 69.79031685028187, 87.3209423093967, 93.2897536009796, 101.65675101367917]
1 : [2.035832446396929, 7.84134306882374, 11.885638675507954, 13.175745534923987, 18.223576879618488]
2 : [2.00130159370868, 5.092676312059547, 5.23001154028977, 10.312829307882781, 13.443780223323833]
3 : [2.0002300995477755, 5.005853911489341, 5.01693816342669, 9.974517981976254, 12.360689791718617]
4 : [2.000050469658806, 5.000576397018797, 5.0021683905289605, 9.763315193639247, 10.924176260526629]
5 : [2.000011730782077, 5.000092449848259, 5.0003363395581575, 8.943461887999756, 10.14709327204672]
6 : [2.0000028054666243, 5.000020014004932, 5.000052904090956, 8.343358510265743, 10.045579442193512]
7 : [2.000000683493345, 5.000004628051813, 5.000008620949716, 8.126027977776271, 10.017094130167454]
8 : [2.0000001686231768, 5.000001076987211, 5.000001553328033, 8.049935189034677, 10.006659859715395]
9 : [2.000000041972391, 5.000000246490849, 5.000000321825083, 8.020938714464442, 10.00261

88 : [2.0000000000422378, 5.000000004672337, 5.00000000508765, 8.000000047998519, 10.000000165870787]
89 : [2.0000000000422378, 5.0000000046722235, 5.000000005087638, 8.000000047998572, 10.000000165870691]
90 : [2.000000000042237, 5.000000004672253, 5.000000005087597, 8.000000047998391, 10.000000165870794]
91 : [2.0000000000422364, 5.000000004672323, 5.0000000050876965, 8.00000004799843, 10.000000165870787]
92 : [2.000000000042237, 5.000000004672284, 5.00000000508762, 8.000000047998352, 10.00000016587079]
93 : [2.0000000000422364, 5.000000004672302, 5.000000005087652, 8.000000047998448, 10.000000165870762]
94 : [2.000000000042237, 5.000000004672341, 5.000000005087696, 8.000000047998425, 10.000000165870707]
95 : [2.0000000000422373, 5.000000004672301, 5.000000005087695, 8.00000004799842, 10.000000165870745]
96 : [2.0000000000422378, 5.00000000467228, 5.0000000050876325, 8.000000047998407, 10.000000165870755]
97 : [2.000000000042236, 5.000000004672309, 5.000000005087659, 8.00000004799846

The multidim-component select in the Visualization dialog box allows to switch between the components of the multidim-GridFunction.